In [49]:
import tkinter as tk
from tkinter import messagebox, ttk
from sqlalchemy import create_engine, text

# Configuração de conexão com o banco de dados
DB_USER = 'postgres'
DB_PASSWORD = '1010'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'DB_Final'

# Mapeamento de dias da semana para números
DIA_SEMANA_MAP = {
    "Domingo": 1,
    "Segunda-feira": 2,
    "Terça-feira": 3,
    "Quarta-feira": 4,
    "Quinta-feira": 5,
    "Sexta-feira": 6,
    "Sábado": 7
}

class HorariosApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Alterar Horários de Atendimento")
        
        # Inicializar listas vazias para médicos e secretários
        self.medicos = []
        self.secretarios = []

        # Dicionário para armazenar os componentes de horário
        self.horarios = {}

        # Criando a interface
        self.criar_interface()

    def connect_db(self):
        try:
            conexao_bd = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
            engine = create_engine(
                conexao_bd,
                connect_args={'options': '-csearch_path=sistema,public'}
            )
            conn = engine.connect()
            return conn
        except Exception as e:
            print(f"Erro ao conectar ao banco de dados: {e}")
            return None

    def buscar_medicos_secretarios(self):
        conn = self.connect_db()
        if conn:
            try:
                # Busca os médicos, incluindo CRM, nome e especialização
                medico_query = text("SELECT ID_Medico, CRM, Nome, Especializacao FROM Medico JOIN Pessoa ON Medico.ID_Pessoa = Pessoa.ID_Pessoa")
                self.medicos = conn.execute(medico_query).fetchall()

                # Busca os secretários, apenas o nome
                secretario_query = text("SELECT ID_Secretario, Nome FROM Secretario JOIN Pessoa ON Secretario.ID_Pessoa = Pessoa.ID_Pessoa")
                self.secretarios = conn.execute(secretario_query).fetchall()

            except Exception as e:
                print(f"Erro ao buscar médicos e secretários: {str(e)}")
            finally:
                conn.close()

    def atualizar_lista_pessoas(self, event):
        """Atualiza a lista de médicos ou secretários no combobox quando o tipo é selecionado."""
        tipo = self.combo_tipo.get()
        self.combo_pessoa['values'] = []

        if tipo == "Médico":
            # Exibe CRM, especialização e nome do médico no ComboBox
            self.combo_pessoa['values'] = [f"CRM: {medico[1]}, {medico[3]} - {medico[2]}" for medico in self.medicos]
        elif tipo == "Secretário":
            # Exibe apenas o nome do secretário
            self.combo_pessoa['values'] = [secretario[1] for secretario in self.secretarios]

        if self.combo_pessoa['values']:
            self.combo_pessoa.current(0)  # Seleciona o primeiro item por padrão

    def criar_horario_componentes(self, dia, num_dia, row):
        """Função para criar as Spinboxes de horário para um dia da semana."""
        label_dia = tk.Label(self.root, text=f"{dia}")
        label_dia.grid(row=row, column=0)

        # Manhã
        self.criar_spinbox(f"manha", row + 1, 0, 0, 12, num_dia)  # Manhã 00:00-11:59
        
        # Tarde
        self.criar_spinbox(f"tarde", row + 2, 0, 12, 18, num_dia)  # Tarde 12:00-17:59
        
        # Noite
        self.criar_spinbox(f"noite", row + 3, 0, 18, 24, num_dia)  # Noite 18:00-23:59

    def criar_spinbox(self, periodo, row, column, hora_inicio_range, hora_fim_range, num_dia):
        """Função para criar Spinboxes de horário."""
        tk.Label(self.root, text=f"{periodo.capitalize()}").grid(row=row, column=column)
        spin_hora_inicio = tk.Spinbox(self.root, values=('-',) + tuple(f"{i:02d}" for i in range(hora_inicio_range, hora_fim_range)), width=4, state="readonly")
        spin_hora_inicio.grid(row=row, column=column + 1)
        spin_minuto_inicio = tk.Spinbox(self.root, values=('-',) + tuple(f"{i:02d}" for i in range(0, 60)), width=4, state="readonly")
        spin_minuto_inicio.grid(row=row, column=column + 2)

        spin_hora_fim = tk.Spinbox(self.root, values=('-',) + tuple(f"{i:02d}" for i in range(hora_inicio_range, hora_fim_range)), width=4, state="readonly")
        spin_hora_fim.grid(row=row, column=column + 3)
        spin_minuto_fim = tk.Spinbox(self.root, values=('-',) + tuple(f"{i:02d}" for i in range(0, 60)), width=4, state="readonly")
        spin_minuto_fim.grid(row=row, column=column + 4)

        # Armazenar as referências das Spinboxes em um dicionário usando um identificador único
        self.horarios[f"{periodo}_hora_inicio_{num_dia}"] = spin_hora_inicio
        self.horarios[f"{periodo}_minuto_inicio_{num_dia}"] = spin_minuto_inicio
        self.horarios[f"{periodo}_hora_fim_{num_dia}"] = spin_hora_fim
        self.horarios[f"{periodo}_minuto_fim_{num_dia}"] = spin_minuto_fim

    def salvar_horarios(self):
        tipo = self.combo_tipo.get()
        id_pessoa = None

        if tipo == "Médico":
            id_pessoa = self.medicos[self.combo_pessoa.current()][0]
        elif tipo == "Secretário":
            id_pessoa = self.secretarios[self.combo_pessoa.current()][0]

        horarios = self.capturar_horarios()
        self.salvar_no_banco(id_pessoa, tipo, horarios)

    def capturar_horarios(self):
        """Captura os horários de todas as Spinboxes e os retorna como dicionário."""
        horarios = {}
        for periodo in ["manha", "tarde", "noite"]:
            for dia, num_dia in DIA_SEMANA_MAP.items():
                # Adicionando o número do dia para o identificador das spinboxes
                hora_inicio = self.horarios.get(f"{periodo}_hora_inicio_{num_dia}", '-').get()
                minuto_inicio = self.horarios.get(f"{periodo}_minuto_inicio_{num_dia}", '-').get()
                hora_fim = self.horarios.get(f"{periodo}_hora_fim_{num_dia}", '-').get()
                minuto_fim = self.horarios.get(f"{periodo}_minuto_fim_{num_dia}", '-').get()

                if hora_inicio != '-' and minuto_inicio != '-' and hora_fim != '-' and minuto_fim != '-':
                    horarios[num_dia] = {
                        'inicio': f"{hora_inicio}:{minuto_inicio}",
                        'fim': f"{hora_fim}:{minuto_fim}"
                    }
        return horarios

    def salvar_no_banco(self, id_pessoa, tipo, horarios):
        conn = self.connect_db()
        if conn:
            try:
                if tipo == "Médico":
                    delete_query = text("DELETE FROM Horario_Atendimento_Medico WHERE ID_Medico = :id_pessoa")
                    conn.execute(delete_query, {"id_pessoa": id_pessoa})

                    insert_query = text("""
                        INSERT INTO Horario_Atendimento_Medico (ID_Dia_Semana, Hora_Inicio_Manha, Hora_Fim_Manha, Hora_Inicio_Tarde, Hora_Fim_Tarde, Hora_Inicio_Noite, Hora_Fim_Noite, ID_Medico)
                        VALUES (:dia_semana, :hora_inicio_manha, :hora_fim_manha, :hora_inicio_tarde, :hora_fim_tarde, :hora_inicio_noite, :hora_fim_noite, :id_pessoa)
                    """)
                    for dia_semana, horario in horarios.items():
                        conn.execute(insert_query, {
                            "dia_semana": dia_semana,
                            "hora_inicio_manha": horario['inicio'] if dia_semana == 1 else None,  # Exemplo de ajuste por dia
                            "hora_fim_manha": horario['fim'] if dia_semana == 1 else None,
                            "hora_inicio_tarde": horario['inicio'] if dia_semana == 2 else None,
                            "hora_fim_tarde": horario['fim'] if dia_semana == 2 else None,
                            "hora_inicio_noite": horario['inicio'] if dia_semana == 3 else None,
                            "hora_fim_noite": horario['fim'] if dia_semana == 3 else None,
                            "id_pessoa": id_pessoa
                        })
                conn.commit()
                messagebox.showinfo("Sucesso", "Horários atualizados com sucesso!")
            except Exception as e:
                conn.rollback()
                messagebox.showerror("Erro", f"Erro ao salvar os horários: {e}")
            finally:
                conn.close()

    def criar_interface(self):
        # Campo de seleção de tipo (Médico ou Secretário)
        tk.Label(self.root, text="Tipo").grid(row=0, column=0)
        self.combo_tipo = ttk.Combobox(self.root, values=["Médico", "Secretário"], state="readonly", width=30)
        self.combo_tipo.grid(row=0, column=1)
        self.combo_tipo.bind("<<ComboboxSelected>>", self.atualizar_lista_pessoas)

        # Campo de seleção de pessoas
        tk.Label(self.root, text="Selecione").grid(row=1, column=0)
        self.combo_pessoa = ttk.Combobox(self.root, state="readonly", width=60)  # Aumentando a largura
        self.combo_pessoa.grid(row=1, column=1)

        # Criar campos de horário para cada dia da semana
        row = 2
        for dia, num_dia in DIA_SEMANA_MAP.items():
            self.criar_horario_componentes(dia, num_dia, row)
            row += 4  # Avança para a próxima linha após os horários de um dia

        # Botão para salvar
        btn_salvar = tk.Button(self.root, text="Salvar", command=self.salvar_horarios)
        btn_salvar.grid(row=row, column=1)

# Inicialização da interface gráfica
root = tk.Tk()
app = HorariosApp(root)
app.buscar_medicos_secretarios()
root.mainloop()
